# Hamiltonian Exponentiation - 5th Place Team Carnivorous Cacti (Tarushii Goel, Kareem Jaber, Cyril Sharma), USA

Below is code and methodology from the fifth place solution to the Spring 2022 Classiq Coding Competition. 


## Submission Description
The main issue we had with solving this problem was just implementing the trotterization algorithm, as it was often difficult to determine sources of error. We would often find very little error in toy examples, but then massive error in the full problem, even when accounting for variables like the increases in the number of pauli strings (10 → 126) or the number of qubits. We discovered bugs such as qiskit’s reversal of qubit order when constructing the circuit unitary (ZIIIIIIIII should apply rotation on the last qubit in the qiskit quantum register, not the first qubit) and forgetting to account for global phase in the error calculation (our circuits would be correct but the matrices would be off by a global phase) which helped us reduce the error. We also developed code to remove higher order error terms by rearranging the terms in the e^iH expansion (https://docs.microsoft.com/en-us/azure/quantum/user-guide/libraries/chemistry/concepts/algorithms, get_circuit_k where k is the order of error). However, we didn’t end up using the more accurate approximations because when we removed the aforementioned bugs, we discovered that we could get below the error requirements with just one iteration (n=1) of the Pauli strings (error=0.0784).

In [35]:
import numpy as np
from numpy.random import default_rng
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.algorithms import NumPyEigensolver
from numpy import linalg as LA
from scipy.linalg import expm, norm

In [49]:
seed = 14
def is_unitary(m):
    return norm(np.eye(len(m)) - m.dot(m.T.conj()))
def is_hermitian(m):
    return norm(m-m.T.conj())
def get_error(e_ith, U_circuit):
    # measure the operator's accuracy
    diff = U_circuit.data - e_ith
    w, v = LA.eig(diff)
    return np.max(np.absolute(w))
def generateBases(numBases, baseLength):
    rng = default_rng(seed)
    bases = []
    gates = ['X', 'Y', 'Z', 'I']
    for _ in range(numBases):
        base = ''.join(rng.choice(gates, baseLength, p=[0.2,0.2,0.2,0.4]))
        bases.append(base)
    return bases
def generateCoefficients(numBases):
    rng = default_rng(seed)
    return rng.uniform(size=len(bases))

In [37]:
coefficients = []
bases = []
with open('LiH-Hamiltonian.txt') as f:
    for line in f:
        x = line.strip().split(" ")
        if (len(line) > 1):
            c = float(x[1])
            c = (c if x[0] == '+' else -c)
            coefficients.append(c)
            bases.append(x[3])

# comment this out when stuff actually works
# numBases = 100
# baseLength = 5
# bases = generateBases(numBases, baseLength)
# coefficients = generateCoefficients(numBases)
hamiltonian = np.zeros((2**len(bases[0]), 2**len(bases[0])), dtype=np.cdouble)
for c, lbl in zip(coefficients, bases):
    op = Operator(Pauli(label=lbl))
    hamiltonian+=c*op.data

In [38]:
def buildPauliString(qc, pauliString, t, n):
    c, b = pauliString
    # implement e^{-i*c*b*t/n}
    b = b[::-1]
    q = []
    # qc.barrier()
    for i, op in enumerate(b):
        if (op == 'X'):
            qc.h(i)
            q.append(i)
        elif (op == 'Z'):
            q.append(i)
        elif (op == 'Y'):
            qc.sdg(i)
            qc.h(i)
            q.append(i)
        elif (op == 'I'):
            continue
    # qc.barrier()
    if (len(q) > 0):
        if (len(q) > 1):
            for i in range(len(q)-1):
                qc.cx(q[i], q[-1])
        qc.rz(2*c*t/n, q[-1])
        if (len(q) > 1):
            for i in reversed(range(len(q)-1)):
                qc.cx(q[i], q[-1])
        # qc.barrier()
        for i, op in enumerate(b):
            if (op == 'X'):
                qc.h(i)
            elif (op == 'Z'):
                continue
            elif (op == 'Y'):
                qc.h(i)
                qc.s(i)
            elif (op == 'I'):
                continue
    # qc.barrier()
    return qc

def buildHamiltonian(qc, pauliStrings, t, n):
    for pauliString in pauliStrings:
        qc = buildPauliString(qc, pauliString, t, n)
    return qc

In [39]:
# first order trotter formulation.
def get_circuit_1(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    for i in range(n):
        qc = buildHamiltonian(qc, pauliStrings, t, n)
    return qc

# second order trotter formulation.
def get_circuit_2(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    reversedPauliStrings = list(zip(reversed(coefficients), reversed(bases)))
    
    for i in range(n):
        qc = buildHamiltonian(qc, pauliStrings, t, n * 2)
        qc = buildHamiltonian(qc, reversedPauliStrings, t, n * 2)
    return qc

# kth order trotter formulation.
def get_circuit_k(t=1, n=1, k=1):
    if k == 1:
        return get_circuit_1(t,n)
    if k == 2:
        return get_circuit_2(t,n)
    elif (k % 2 == 0):
        s_k = (4 - (4**(1 / (k - 1))))**(-1)
        c1 = get_circuit_k(s_k * t, n, k-2)
        c1_2 = c1.compose(c1)
        c2 = get_circuit_k((1 - 4 * s_k) * t, n, k-2)
        c = c1_2.compose(c2).compose(c1_2)
        return c
    else:
        return "make k even!!"

def get_unitaries_k(t=1, n=1, k=1):
    e_ith = expm(-1.j * t * hamiltonian)
    e_ith_norm = e_ith / e_ith[0,0]
    qc = get_circuit_k(t=t, n=n, k=k)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    U_circuit_norm = U_circuit.data / U_circuit.data[0,0]
    return e_ith_norm, U_circuit_norm

In [50]:
e_ith5, U_circuit5 = get_unitaries_k(t=1, n=1, k=1)
get_error(e_ith5, U_circuit5)

0.07845989127970808

In [ ]:
qc = get_circuit_k(t=1, n=1, k=1)
qc.draw()

In [43]:
aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'], optimization_level=3)
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())
qc_transpiled.qasm(filename='hamiltonian1.qasm')

1411
OrderedDict([('u', 1105), ('cx', 914)])


'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[10];\nu(pi/2,0,pi/2) q[0];\nu(pi/2,0,pi/2) q[1];\nu(pi/2,-pi/2,pi/2) q[5];\nu(pi/2,0,pi/2) q[6];\ncx q[0],q[6];\nu(pi/2,1.4998704,-pi) q[0];\ncx q[1],q[6];\nu(0,0.96224849,-0.96224849) q[1];\nu(3.0740219,pi/2,-pi) q[6];\ncx q[5],q[6];\nu(3.1355233,-pi,-pi/2) q[5];\nu(1.5707951,1.5703865,-1.5768518) q[6];\ncx q[5],q[6];\nu(pi,0.93890883,2.5097052) q[5];\nu(pi/2,-0.067570782,-pi) q[6];\ncx q[1],q[6];\nu(pi/2,-pi/2,pi/2) q[1];\nu(1.6383671,-pi/2,pi/2) q[6];\ncx q[0],q[6];\nu(pi/2,0,-3.0706667) q[0];\nu(1.6383671,pi/2,-pi/2) q[6];\ncx q[1],q[6];\nu(0,0.96224849,-0.96224849) q[1];\nu(3.0740219,pi/2,-pi) q[6];\ncx q[5],q[6];\nu(3.1355233,-pi,-pi/2) q[5];\nu(1.5707951,1.5703865,-1.5768518) q[6];\ncx q[5],q[6];\nu(pi,-0.25710174,-0.25710174) q[5];\nu(pi/2,-0.067570782,-pi) q[6];\ncx q[1],q[6];\ncx q[0],q[6];\nu(pi/2,pi/2,-pi/2) q[0];\nu(pi/2,pi/2,-pi/2) q[1];\nu(pi/2,0,pi) q[6];\nu(pi/2,0,pi/2) q[9];\ncx q[0],q[9];\nu(pi/2,1.4998704,-pi) q[0];\ncx 